# Computerphysik Programmiertutorial 13
Prof. Dr. Matteo Rizzi und Dr. Markus Schmitt - Institut für Theoretische Physik, Universität zu Köln
&nbsp;

**Github**: [https://github.com/markusschmitt/compphys2022](https://github.com/markusschmitt/compphys2022)

**Inhalt dieses Notebooks**: Metropolis-MCMC Simulation des Potts Modells, Burn-in und Autokorrelation

# Potts-Modell

Das Potts-Modell ist eine Verallgemeinerung des Ising-Modells. 

Im Ising-Modell kann jeder Mikro-Maget zwei Ausrichtungen annehmen: 

* "hoch" $\uparrow$ oder "runter" $\downarrow$. 

Wir verallgemeinern das nun im Potts-Modell zu Mikro-Magneten, die in drei mögliche Richtungen zeigen können: 

* "oben" $\uparrow$, "unten rechts" $\searrow$ oder "unten links" $\swarrow$

Wir weisen den drei möglichen Ausrichtungen im folgenden die numerischen Werte $s=0,1,2$ zu, die den Winkeln $\theta_n=\frac{2\pi n}{3}$ entsprechen.

Analog zum Ising-Modell platzieren wir unsere Potts-Mikro-Magnete $\vec s$ auf einem Quadratgitter und definieren die Energie-Funktion

$$
E(\vec s) = -\sum_{\langle i,j\rangle}\cos\Big(\frac{2\pi}{3} (s_i-s_j)\Big)
= -\frac{1}{2}\sum_i\sum_{j\in\mathcal N(i)}\cos\Big(\frac{2\pi}{3} (s_i-s_j)\Big)
\equiv\frac12\sum_i E_{i}(\vec s)
$$

Die Energie wird also minimiert, wenn alle Mikro-Magnete $s_i$ parallel ausgerichtet sind.

Wir wollen in diesem Tutorial das Temperaturabhängige Verhalten des Potts-Modells numerisch untersuchen. Die Basis dafür ist die *Boltzmann-Verteilung*, laut der bei einer Temperatur $T$ die Wahrscheinlichkeit, dass sich das System in der Konfiguration $\vec s$ befindet, durch

$$
p_T(\vec s)\propto \exp\big(-E(\vec s)/T\big)
$$

gegeben ist (wir setzen hier die Naturkonstante $k_B=1$).

Wie in der Vorlesung besprochen, können mit dem **Metropolis-Algorithmus** eine Menge von Konfigurationen $\vec s_n$ erzeugen, in der jedes Element $\vec s_n$ mit der Wahrscheinlichkeit $p_T(\vec s_n)$ auftaucht. Sobald wir diese Konfigurationen haben, können wir Erwartungswerte bezüglich der Verteilung $p_T(\vec s)$ schätzen.

Der Metropolis-Algorithmus basiert auf einem Markov-Prozess auf den Konfigurationen $\vec s$, in dem ausgehend von $\vec s$ eine zufällige neue Konfiguration $\vec s'$ vorgeschlagen und mit der Wahrscheinlichkeit

$$
p_{\text{akz.}}^T(\vec s',\vec s)=\text{min}\big\{e^{-[E(\vec s')-E(\vec s)]/T}, 1\big\}
$$

angenommen wird.

In [ ]:
using Plots
using Random
using LaTeXStrings

## Implementierung des Potts-Modells

Die folgende Funktion generiert eine zufällige Konfiguration von Potts-Mikro-Magneten ($s_j\in\{0,1,2\}$) auf einem Quadratgitter der größe $L\times L$:

In [ ]:
function random_config(L)
    return config = Int.(ceil.(3*rand(L,L))).-1
end

In [ ]:
heatmap(random_config(5), border=:none, legend=:none, aspect_ratio=:equal)

Wir werden im Folgenden die Konvention nutzen, dass die Nachbarn "oben", "unten", "links", und "rechts" mit 1,2,3,4 durchnummeriert sind. Die Funktion `neighbor` soll dann für den Gitterplatz $i,j$ in einem $L\times L$-Gitter die Gitterindizes des $k$-ten Nachbarn zurückgeben:

In [ ]:
function neighbor(i,j,k,L)
    # k= 1 - oben, 2 - unten, 3 - links, 4 - rechts
    if k==1
        return 1+(L+i-2)%L, j
    end
    if k==2
        return 1+i%L, j
    end
    if k==3
        return i, 1+(L+j-2)%L
    end
    if k==4
        return i, 1+j%L
    end
end

Test der `neighbor`-Funktion:

In [ ]:
i,j=1,1
println("$i,$j -> oben: ", neighbor(i,j,1,13))
println("$i,$j -> unten: ", neighbor(i,j,2,13))
println("$i,$j -> links: ", neighbor(i,j,3,13))
println("$i,$j -> rechts: ", neighbor(i,j,4,13))

Als nächstes brauchen wir eine Funktion, mit der wir einen zufälligen Vorschlag für $\vec s'$ generieren können. Eine einfache Wahl ist ausgehend von einer Konfiguration $\vec s$ einen einzelnen Mikro-Magneten im Gitter auszuwählen und seine Richtung zu ändern. Dafür schreiben wir eine Funktion, die die Koordinaten eines zufälligen Gitterplatzes zurückgibt und einen zufälligen Wert, um den die Richtung des Mikro-Magneten gedreht werden soll:

In [ ]:
function propose_update(L)
    
    # zufälliger Gitterplatz
    i = Int(ceil(L*rand()))
    j = Int(ceil(L*rand()))
    
    # zufälliger Drehwinkel
    s = Int(ceil(2*rand()))
    
    return i,j,s
    
end

In [ ]:
propose_update(10)

Da sich in unserem Fall die Konfigurationen $\vec s$ und $\vec s'$ bei der Berechnung der Energiedifferenz für die Akzeptanzwahrscheinlichkeit nur in einem Gitterplatz unterscheiden werden, reicht es statt der Energiedifferenz die Differenz der lokalen Enerigen

$$
E_{i}(\vec s)=-\sum_{j\in\mathcal N(i)}\cos\Big(\frac{2\pi}{3} (s_i-s_j)\Big)
$$

zu berechnen. Das macht die folgende Funktion:

In [ ]:
function site_energy(i,j,config)
    L = size(config)[1]
    
    E = 0
    
    theta = 2*pi/3
    
    for k in 1:4
        ni, nj = neighbor(i,j,k,L)
        E -= cos((config[i,j] - config[ni,nj]) * theta)
    end
    
    return E
end

Außerdem möchten wir gerne die Gesamtenergie berechnen können:

In [ ]:
function energy(config)
    L = size(config)[1]
    
    E=0
    for i in 1:L
        for j in 1:L
            E += site_energy(i,j,config)
        end
    end
    
    return 0.5*E
end

## Metropolis Algorithmus

Der Kern des Metropolis-Algorithmus ist der Update-Schritt, in dem eine neue Konfiguration vorgeschlagen und dann entsprechend der Akzeptanzwahrscheinlichkeit angenommen oder verworfen wird:

In [ ]:
function step!(config, T)
    L = size(config)[1]
    

end

Zwischen zwei Konfigurationen, die wir später zur Berechnung von Observablen verwenden, sollte immer eine Reihe von Update-Schritten liegen (wir werden weiter unten sehen warum). Der Prozess dieser Folge von Updates wird oft *Sweep* genannt. Die folgende Funktion führt einen Sweep mit einer Gewunschten Zahl an Update-Schitten durch:

In [ ]:
function sweep!(config, T, sweep_steps)

end

Nun haben wir alle Zutaten um den eigentlichen Metropolis-Algorithmus zu schreiben:

**Parameter:**
* `L` Kantenlänge des Quadratgitters
* `T` Temperatur
* `num_samples` Zahl der Konfigurationen, die generiert werden sollen
* `sweep_steps` Zahl der Update-Schritte zwischen berücksichtigten Konfigurationen
* `burn_in_steps` Zahl der Update-Schritte vor der ersten berücksichtigten Konfiguration

In [ ]:
function mcmc(L, T, num_samples, sweep_steps, burn_in_steps)
    # zufällige Anfangskonfiguration
    config = random_config(L)

    # Container, der mit Konfigurationen gefüllt wird
    samples = zeros(Int, (L,L,num_samples))
    
    # "Burn-in": Einige Updates am Anfang um die stationäre Region zu erreichen
    for n in 1:burn_in_steps
        step!(config, T)
    end
    
    # Sammeln von Konfigurationen
    for n in 1:num_samples
        sweep!(config, T, sweep_steps)
        
        samples[:,:,n] = config
    end
    
    return samples
end

In [ ]:
configs = mcmc(10, 5, 100, 1, 0)

In [ ]:
heatmap(configs[:,:,67], border=:none, legend=:none, aspect_ratio=:equal)

## Simulation

Nun können wir das Potts-Modell bei verschiedenen Temperaturen simulieren.

Wir möchten gerne die Temperaturabhängigkeit der spezifischen Wärme

$$
C_V(T)=\frac{1}{T}\Big(\langle E^2\rangle_T-\big(\langle E\rangle_T\big)^2\Big)
$$

bestimmen. Hier ist $\langle X\rangle_T$ der Erwartungswert der größe $X(s)$ bezüglich der Boltzmann-Verteilung $p_T(\vec s)$.

In [ ]:
using Statistics

# Hilfsfunktion, die die Funktion `f` auf der Liste von Konfigurationen `configs` auswertet
function eval(f, configs)
    return [f(configs[:,:,j]) for j in 1:size(configs)[3]]
end

Random.seed!(1234)

L = 10 # Systemgröße
num_samples = 10000 # Anzahl von Konfigurationen
sweep_steps = L*L 
burn_in_steps = 10000
Ts = range(0.5,5,step=0.25) # Temperaturbereich

cv = []

for T in Ts
    configs = mcmc(L, T, num_samples, sweep_steps, burn_in_steps)
    push!(cv, var(eval(energy, configs))/T^2)
end

plot(Ts, cv)
xlabel!("Temperatur")
ylabel!("Spezifische Wärme")

In [ ]:
L=20
T=1.75;

# Stolpersteine in MCMC

Für verlässliche Ergebnisse ist eine gute Wahl der Parameter `sweep_steps` und `burn_in_steps` essenziell.

## Burn-in

Die Anfangskonfiguration ist in diesen Simulationen beliebig gewählt und kann in einer Region mit sehr kleinen Boltzmann-Gewichten liegen.

Was passiert, wenn wir keine "Burn-in" Schritte machen und `sweep_steps=1` setzen?

In [ ]:
configs = mcmc(L, T, 20000, 1, 0)

E = eval(energy, configs)

plot(E)
xlabel!(L"Metropolis-Schritt $n$")
ylabel!(L"Energie $E(\vec s_n)$")

Die Energien am Anfang der Simulation haben nichts mit dem späteren Gleichgewichtswert zu tun. Wir sollten daher genügend "Burn-in" Schritte machen.

In [ ]:
configs = mcmc(L, T, 20000, 1, 5000)

E = eval(energy, configs)

plot(E)
xlabel!(L"Metropolis-Schritt $n$")
ylabel!(L"Energie $E(\vec s_n)$")

## Autokorrelation / Autokorrelationszeit

Im Bild oben ist zu sehen, dass aufeinanderfolgende Konfigurationen sich ähnlicher sind als Korrelationen mit größerem "zeitlichen" Abstand. Diese Eigenschaft wird mit der Autokorrelationsfunktion quantifiziert:

$$
C_X(t)=\big\langle(X_n-\langle X\rangle)(X_{n+t}-\langle X\rangle)\big\rangle=\langle X_nX_{n+t}\rangle-\langle X\rangle^2
$$

Hier bezeichnet $\langle\cdot\rangle$ den Mittelwert über unsere Datenreihe.

In [ ]:
function autocorrelation(X, tmax)
    C = zeros(tmax+1)
    Xmean = mean(X)
    Xnrm = X .- Xmean
    
    for t in 0:tmax
        C[t+1] = mean(Xnrm[1:end-t] .* Xnrm[1+t:end])
    end
    
    return C
end

In [ ]:
plot(autocorrelation(E, 500))
xlabel!(L"Abstand $t$")
ylabel!(L"Autokorrelation $C_E(t)$")

In [ ]:
configs = mcmc(L, T, 1000, 5000, 5000)

E = eval(energy, configs)

plot(E)
xlabel!(L"Metropolis-Schritt $n$")
ylabel!(L"Energie $E(\vec s_n)$")

In [ ]:
plot(autocorrelation(E, 500))
xlabel!(L"Abstand $t$")
ylabel!(L"Autokorrelation $C_E(t)$")

### Warum ist Autokorrelation ein Problem?

Wenn wir mit korrelierten Konfigurationen arbeiten, schätzen wir leicht die Qualität der Ergebnisse falsch ein!

Der Standardfehler eines Mittelwerts wird angegeben als die Standardabweichung geteilt durch die Wurzel aus der Zahl der Verwendeten Konfigurationen:

$$
\Delta_X=\frac{\sigma_X}{\sqrt{N_{MC}}}
$$

Testen wir die Fehlerangaben mit verschiedenen Werten für `sweep_steps`:

In [ ]:
num_samples=1000
correlated_configs = mcmc(20, 1.75, num_samples, 1, 5000)
correlated_E = mean(eval(energy, correlated_configs))
correlated_E_err = std(eval(energy, correlated_configs))/sqrt(num_samples)

good_configs = mcmc(20, 1.75, num_samples, 10000, 5000)
good_E = mean(eval(energy, good_configs))
good_E_err = std(eval(energy, good_configs))/sqrt(num_samples)

println("Korrelierte Energie: $correlated_E +/- $correlated_E_err")
println("Gute Energie: $good_E +/- $good_E_err")

Sind diese Fehlerbalken angemessen?

In [ ]:
corr=[]
corr_err=[]
good=[]
good_err=[]

for n = 1:10
    correlated_configs = mcmc(20, 1.75, num_samples, 1, 5000)
    correlated_E = mean(eval(energy, correlated_configs))
    correlated_E_err = std(eval(energy, correlated_configs))/sqrt(num_samples)

    good_configs = mcmc(20, 1.75, num_samples, 10000, 5000)
    good_E = mean(eval(energy, good_configs))
    good_E_err = std(eval(energy, good_configs))/sqrt(num_samples)
    
    push!(corr, correlated_E)
    push!(corr_err, correlated_E_err)
    push!(good, good_E)
    push!(good_err, good_E_err)
end

In [ ]:
plot(collect(1:10), corr, yerr=corr_err, label="korreliert")
plot!(collect(1:10), good, yerr=good_err, label="unkorreliert")
xlabel!("Realisierung")
ylabel!(L"Mittelwert $\langle E\rangle_T$")

Wenn mit MCMC erzeugte Konfigurationen **nicht unkorreliert** sind, muss man von einer **effektiv reduzierten Zahl an Konfigurationen** ausgehen:

$$
\tilde N_{MC}=\frac{N_{MC}}{\tau_C}
$$

wobei $\tau_C$ die **Autokorrelationszeit** ist, die angibt wie viele aufeinanderfolgende Konfigurationen signifikant korreliert sind.

## Was passiert im Potts-Modell bei $T\approx1.5$?

In [ ]:
using Printf

L=100

config = random_config(L)

T0=2.5
Ts = range(T0,0.5,step=-0.05)

for n in 1:10
    sweep!(config, T0, L^2)
end
config0 = copy(config)

# pygui(true)
# axis("off")
anim = @animate for T in Ts
    for n in 1:20
        sweep!(config, T, L^2)
    end
    
    heatmap(config, border=:none, legend=:none, aspect_ratio=:equal, title=@sprintf("T = %.2f", T))
#     title(@sprintf("T = %.2f", T))
#     show()
#     sleep(0.1)
end
# pygui(false);

gif(anim, "potts_cooling.gif", fps = 5)

In [ ]:

p1 = heatmap(config0, border=:none, legend=:none, aspect_ratio=:equal, title="T=2.5")
p2 = heatmap(config, border=:none, legend=:none, aspect_ratio=:equal, title="T=0.5")
plot(p1,p2, size=(600,300))